In [281]:
"""
скрипт для нахождения совпадения между школами из 2 перечней
"""

'\nскрипт для нахождения совпадения между школами из 2 перечней\n'

In [282]:
import pandas as pd
import time
import re

In [283]:
def extract_name(cell):
    result = re.search(r'«(.+)»',cell)
    if result:
        return result.group(1)
    else:
        return cell

In [284]:
def extract_name_cavs(cell):
    
    result = re.search(r'\"(.+)\"',cell)
    if result:
        return result.group(1)
    else:
        return cell

In [285]:
def replace_soh(cell):
    result = re.sub('средняя общеобразовательная школа','СОШ',cell)
    result = re.sub('Cредняя общеобразовательная школа','СОШ',result)
    return result


In [286]:
def replace_ooh(cell):
    result = re.sub('основная общеобразовательная школа','ООШ',cell)
    result = re.sub('Основная общеобразовательная школа','ООШ',result)
    return result

In [287]:
def replace_kav(cell):
    """
    убрать кавычки
    """
    cell = cell.replace('«','')
    cell = cell.replace('»','')
    cell = cell.replace('"','')
    return cell


In [288]:
df_414 = pd.read_excel('data/Копия 414 перечень.xlsx')
df_192 = pd.read_excel('data/192 Педагоги-навигаторы Бурятия.xlsx',sheet_name='Районы',usecols='C:F')

In [289]:
# обработка соответствия должности 

In [290]:
df_192['Должность'] = df_192['Должность'].fillna('')

In [291]:
df_192['Должность'] = df_192['Должность'].apply(lambda x:x.lower())

In [292]:
df_192['Статус должности'] = df_192['Должность'].str.contains('зам')|  df_192['Должность'].str.contains('дир')

In [293]:
df_192['Статус должности'] = df_192['Статус должности'].apply(lambda x:'Соответствует' if x else 'Не соответствует')

In [294]:
# работа со школами

In [295]:
df_192['Название'] = df_192['Наименование ОО'].apply(extract_name)

In [296]:
df_192['Название'] = df_192['Название'].apply(extract_name_cavs)

In [297]:
df_192.head()

,Наименование ОО,Ф.И.О,Должность,Контактный телефон,Статус должности,Название
0,МБОУ «Баргузинская СОШ»,Шелковникова Мария Сергеевна,зам по вр,89246548422,Соответствует,Баргузинская СОШ
1,МБОУ «Юбилейная СОШ»,Ахмадулина Дина Александровна,зам по вр,89243919123,Соответствует,Юбилейная СОШ
2,МБОУ «Читканская СОШ»,Белевич Екатерина Валерьевна,зам по вр,89221169651,Соответствует,Читканская СОШ
3,МБОУ «Уринская СОШ»,Слесарева Наталья Игоревна,педагог-психолог,89244548880,Не соответствует,Уринская СОШ
4,"МБОУ ""Улюнская СОШ им. С. Хамнаева""",Цыденова Марина Баторовна,"учитель начальных классов, педагог-психолог",79243935471,Не соответствует,Улюнская СОШ им. С. Хамнаева


In [298]:
# заменяем названия 
dct_replace = {'Средняя общеобразовательная школа':'СОШ','средняя общеобразовательная школа':'СОШ','основная общеобразовательная школа':'ООШ'}

In [299]:
df_414['Полное наименование образовательной организации'] = df_414['Полное наименование образовательной организации'].fillna('')

In [300]:
df_414['Название'] = df_414['Полное наименование образовательной организации'].apply(extract_name)

In [301]:
df_414['Название'] = df_414['Название'].apply(replace_soh)
df_414['Название'] = df_414['Название'].apply(replace_ooh)


In [302]:
df_414['Название'] 

0        Усть-Баргузинская СОШ им. К.М. Шелковникова
1                                     Адамовская ООШ
2                                   Баргузинская СОШ
3                                   Баянгольская СОШ
4                                    Гусихинская ООШ
                           ...                      
446                            ООШ №15 г. Улан – Удэ
447                                                 
448                 Открытая сменная № 1 г. Улан-Удэ
449            Российская гимназия №59 г. Улан – Удэ
450    Музыкально – гуманитарный лицей им Д. Аюшеева
Name: Название, Length: 451, dtype: object

In [303]:
t = time.localtime()
current_time =time.strftime('%H_%M_%S', t)

In [304]:
df_192.to_excel(f'data/Соответсвие должности {current_time}.xlsx',index=False)

In [305]:
outer_df = df_414.merge(df_192,left_on='Название',right_on='Название',how='outer',indicator=True)

In [306]:
outer_df[outer_df['_merge'] == 'both']

,Unnamed: 0,Муниципальное образование,Полное наименование образовательной организации,Уровень,"Информация о сотруднике образовательной организации, ответственного за реализацию мероприятий профориентационного минимума ФИО (полностью)","Участник проекта ""Билет в будущее""",Название,Наименование ОО,Ф.И.О,Должность,Контактный телефон,Статус должности,_merge
2,3.,Баргузинский район,Муниципальное бюджетное общеобразовательное уч...,основной,Суровая Ольга Геннадьевна,Да,Баргузинская СОШ,МБОУ «Баргузинская СОШ»,Шелковникова Мария Сергеевна,зам по вр,89246548422,Соответствует,both
9,9.,Баргузинский район,Муниципальное бюджетное общеобразовательное уч...,основной,Ахмадулина Нина Вячеславовна,Да,Уринская СОШ,МБОУ «Уринская СОШ»,Слесарева Наталья Игоревна,педагог-психолог,89244548880,Не соответствует,both
11,11.,Баргузинский район,Муниципальное бюджетное общеобразовательное уч...,основной,Мазур Вера Валерьевна,Да,Читканская СОШ,МБОУ «Читканская СОШ»,Белевич Екатерина Валерьевна,зам по вр,89221169651,Соответствует,both
12,12.,Баргузинский район,Муниципальное бюджетное общеобразовательное уч...,основной,Очирова Светлана Вячеславовна,Да,Юбилейная СОШ,МБОУ «Юбилейная СОШ»,Ахмадулина Дина Александровна,зам по вр,89243919123,Соответствует,both
14,14.,Баунтовский эвенкийский район,Муниципальное автономное учреждение «Багдаринс...,основной,Ковалева Наталья Александровна,Да,Багдаринская СОШ,"МАОУ ""Багдаринская СОШ""",Артемьева Ирина Николаевна,зам директора по воспитательной работе,89247787415,Соответствует,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...
367,327.,Тункинский район,Муниципальное общеобразовательное учреждение «...,основной,Тюменцева светлана Александровна,NaN,Тункинская СОШ,"МБОУ ""Тункинская СОШ""",Сороковикова Елена Ивановна,зам. по увр,89024526641,Соответствует,both
372,332.,Хоринский район,Муниципальное бюджетное общеобразовательное уч...,основной,Дабаев Жамсаран Ширапович,NaN,Баянгольская ООШ,МБОУ «Баянгольская ООШ»,Дабаев Жамсаран Ширапович,директор,89247539406,Соответствует,both
376,336.,Хоринский район,Муниципальное бюджетное общеобразовательное уч...,основной,Дамбиева Мария Владимировна,NaN,Зун-Хурайская ООШ,МБОУ «Зун-Хурайская ООШ»,Эпова Евгения Андреевна,учитель,89246556562,Не соответствует,both
378,338.,Хоринский район,Муниципальное бюджетное общеобразовательное уч...,основной,Шурыгина Ирина Александровна,NaN,Санномыская СОШ,МБОУ «Санномыская СОШ»,Котова Татьяна Юрьевна,учитель русского языка и литературы,89503823258,Не соответствует,both


In [307]:
outer_df.drop(columns='Unnamed: 0',inplace=True)

In [308]:
dc = {'left_only':'Только в 414 перечне','right_only':'Только в 192 Педагоги навигаторы','both':'В обоих списках'}

In [309]:
outer_df['_merge'] = outer_df['_merge'].replace(dc)

In [310]:
outer_df.to_excel(f'data/Список.xlsx {current_time}.xlsx',index=False)